# Inferenza bayesiana {#sec-bayes-inference}

**Prerequisiti**

- Leggere il capitolo *Bayes' Rule* di @Johnson2022bayesrules.

**Concetti e Competenze Chiave**

- Distribuzione marginale.
- Approccio analitico e numerico per determinare la distribuzione a posteriori.
- Linguaggi di programmazione probabilistici.
- Inferenza predittiva.

**Preparazione del Notebook**

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import arviz as az
import scipy.stats as stats
from scipy.stats import beta
from scipy.special import factorial
from scipy.special import comb
import scipy.special as sp
from scipy.stats import beta
from scipy.integrate import quad

In [2]:
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")
%config InlineBackend.figure_format = 'retina'

## Introduzione {.unnumbered}

Questo capitolo approfondisce i concetti introdotti nel capitolo precedente, presentando l'aggiornamento bayesiano in modo più formale e dettagliato.

## Paradigma Bayesiano

L'approccio bayesiano alla statistica si fonda sull'idea di rappresentare la conoscenza a priori sui parametri che governano un fenomeno attraverso distribuzioni di probabilità. Queste distribuzioni a priori riflettono le credenze iniziali del ricercatore riguardo ai parametri, prima di osservare i dati. Quando nuovi dati vengono raccolti, l'informazione fornita da tali dati viene integrata nel modello tramite la funzione di verosimiglianza, che rappresenta la probabilità di osservare quei dati dati i parametri ipotizzati.

Attraverso l'applicazione del Teorema di Bayes, le credenze a priori vengono aggiornate combinando la distribuzione a priori con la verosimiglianza dei dati. Questo processo produce la distribuzione a posteriori, che rappresenta una nuova e più informata stima dei parametri, tenendo conto sia delle credenze iniziali sia dei dati osservati.

L'approccio bayesiano richiede un cambiamento di prospettiva rispetto ai metodi classici di stima dei parametri. Non ci si limita più a trovare un singolo valore "ottimale" per i parametri del modello. Invece, l'obiettivo è determinare l'intera distribuzione a posteriori dei parametri, che descrive in modo completo lo stato di conoscenza attuale. Solo questa distribuzione fornisce una rappresentazione adeguata dell'incertezza associata ai parametri, permettendo di quantificare non solo quali valori sono più probabili, ma anche l'ampiezza dell'incertezza su tali stime.

### Vantaggi dell'Approccio Bayesiano

Un aspetto distintivo dell'inferenza bayesiana è la sua capacità di gestire l'incertezza in modo esplicito. Invece di limitarsi a una singola stima puntuale, l'approccio bayesiano considera l'intero spettro di valori possibili per i parametri e le loro rispettive probabilità. Questo consente una rappresentazione più ricca delle informazioni disponibili e una valutazione più robusta delle ipotesi.

Inoltre, l'approccio bayesiano è altamente flessibile, permettendo di incorporare informazioni precedenti sotto forma di distribuzioni a priori. In contesti in cui si dispone di conoscenze pregresse, come dati di studi precedenti o teorie consolidate, questa caratteristica offre un vantaggio notevole rispetto agli approcci frequentisti, che non integrano facilmente tali informazioni.

In conclusione, il paradigma bayesiano offre una visione più ampia e completa dell'incertezza e della variabilità dei parametri rispetto ai metodi tradizionali, rappresentando un quadro teorico e pratico fondamentale per l'inferenza statistica e la modellazione.

## Densità di Probabilità

Nei capitoli precedenti abbiamo esaminato alcuni esempi di <font color='orange'>funzioni di densità di probabilità (PDF)</font>. Ma quali sono le caratteristiche generali di una PDF?

Se $X$ è una variabile casuale con una funzione di densità di probabilità $p(x)$, la probabilità che $X$ assuma un valore nell'intervallo $(a, b)$ può essere calcolata come:

$$
p(X \in (a,b)) = \int_a^b p(x)dx.
$$

Per variabili discrete, l'integrazione si trasforma in una somma.

### Le Regole di Somma e Prodotto

Date due variabili casuali continue $x$ e $y$, le regole di somma e prodotto per le densità di probabilità si esprimono come:

$$
\begin{align*}
p(y) = \int p(x,y)dx \quad &\text{- regola della somma},\\
p(x,y) = p(y|x) p(x) = p(x|y) p(y) \quad &\text{- regola del prodotto}.
\end{align*}
$$

La probabilità $p(y)$ è chiamata <font color='orange'>probabilità marginale</font>.

La regola del prodotto specifica che la distribuzione congiunta di due variabili può essere espressa come il prodotto di una distribuzione condizionata $p(y \mid x)$ e una distribuzione marginale $p(x)$, o viceversa.

### La Distribuzione Marginale

La distribuzione marginale si riferisce alla distribuzione di probabilità di una variabile quando si tiene conto di tutte le possibili variazioni dell'altra variabile in una distribuzione congiunta. In altre parole, essa descrive la probabilità di una variabile indipendentemente dall'altra.

Consideriamo due variabili correlate, $x$ e $y$. Possiamo esprimere la relazione tra di esse con la <font color='orange'>regola del prodotto</font>:

$$
p(x, y) = p(y \mid x)p(x),
$$

dove $p(y \mid x)$ è la probabilità di $y$ dato un certo valore di $x$, e $p(x)$ è la distribuzione di probabilità di $x$.

Per ottenere la distribuzione marginale di $y$, dobbiamo sommare o integrare $p(y \mid x)$ su tutti i possibili valori di $x$:

$$
p(y) = \int p(y \mid x)p(x)dx.
$$

In questo modo, la distribuzione marginale di $y$ rappresenta la probabilità di $y$, tenendo conto di tutte le possibili variazioni di $x$.

### Il Teorema di Bayes

Dalla regola di prodotto, e sfruttando la proprietà di simmetria $p(x \mid y)p(y) = p(y \mid x)p(x)$, deriviamo immediatamente la regola di Bayes:

$$
p(y \mid x) = \frac{p(x \mid y)p(y)}{p(x)} = \frac{p(x \mid y)p(y)}{\int p(x \mid y)p(y)dy}.
$$

Questa formula è l'elemento chiave nell'inferenza bayesiana, poiché definisce la densità a posteriori di $y$, $p(y \mid x)$, dopo aver incorporato l'informazione $x$ attraverso il modello di probabilità condizionata $p(x \mid y)$. La probabilità marginale di $x$, $p(x)$, funge da costante di normalizzazione, garantendo che $p(y \mid x)$ sia una corretta funzione di densità di probabilità.

### Modellizzazione e Inferenza Bayesiana

La modellizzazione bayesiana consiste nel descrivere matematicamente tutti i dati osservabili $y$ e i parametri non osservabili, detti anche parametri "latenti" $\theta$, definendo la distribuzione congiunta di dati e parametri $p(y, \theta)$.

Si costruiscono modelli probabilistici per le quantità osservate condizionate ai parametri $p(y \mid \theta)$ e per le quantità non osservate, rappresentate dalla distribuzione a priori $p(\theta)$, che rappresenta le nostre conoscenze precedenti sui parametri. Questi due elementi vengono combinati, seguendo la regola del prodotto, per formare una distribuzione congiunta:

$$
p(y, \theta) = p(y \mid \theta)p(\theta).
$$

### Il Modello Osservazionale

La funzione

$$
p(y \mid \theta)
$$

è un <font color='orange'>modello probabilistico dei dati osservati</font> che mette in relazione $y$ con i parametri sconosciuti $\theta$ che vogliamo stimare. Questo modello rappresenta l'evidenza fornita dai dati e costituisce la principale fonte di informazione. In questo contesto, viene chiamato <font color='orange'>funzione di verosimiglianza</font> (*likelihood*). È importante notare che la funzione di verosimiglianza nel contesto bayesiano non è diversa da quella utilizzata nell'approccio frequentista: in entrambi i casi collega i dati osservati ai parametri sconosciuti.

### La Distribuzione a Priori

La distribuzione

$$
p(\theta)
$$

rappresenta la <font color='orange'>distribuzione a priori</font> dei parametri, che codifica le conoscenze preesistenti sui parametri stessi. Questa distribuzione a priori può essere <font color='orange'>informativa</font> o <font color='orange'>non informativa</font>, a seconda della quantità di informazioni affidabili che si possiedono sui parametri. Uno degli aspetti principali che differenziano l'approccio bayesiano da quello frequentista è l'uso delle distribuzioni di probabilità per i parametri sconosciuti. Queste distribuzioni vengono poi combinate con la funzione di verosimiglianza per ottenere una distribuzione a posteriori, che incorpora sia le informazioni precedenti che le evidenze fornite dai nuovi dati.

### Inferenza sui Parametri

Ottenere la distribuzione a posteriori dei parametri sconosciuti è l'elemento centrale dell'approccio bayesiano. Riformulando la regola di Bayes in termini di $y$ e $\theta$, otteniamo una formula che ci mostra come calcolare la distribuzione a posteriori:

$$
p(\theta \mid y) = \frac{p(y \mid \theta)p(\theta)}{p(y)} = \frac{p(y \mid \theta)p(\theta)}{\int p(y \mid \theta)p(\theta) d\theta}.
$$

Il denominatore della regola di Bayes,

$$
p(y) = \int p(y \mid \theta)p(\theta) d \theta,
$$

è chiamato <font color='orange'>verosimiglianza marginale</font>, poiché integra la verosimiglianza rispetto all'informazione a priori sui parametri. Questa quantità è anche nota come <font color='orange'>evidenza</font> del modello e serve a normalizzare la distribuzione a posteriori, rendendola una vera distribuzione di probabilità. L'inferenza finale sarà un compromesso tra l'evidenza fornita dai dati e l'informazione a priori disponibile.

### Inferenza bayesiana in sintesi: verosimiglianza, prior, posteriore

Per riassumere, ecco tutti i componenti fondamentali dell'inferenza bayesiana.

Il teorema di Bayes, espresso in termini di dati $y$ e parametri del modello $\theta$, è

$$
p(\theta \mid y) = \frac{p(y \mid \theta)p(\theta)}{p(y)},
$$

dove:

- Il denominatore $p(y)$ è la costante di normalizzazione o *evidenza*.
- $p(\theta)$ rappresenta il *prior*, ovvero le credenze iniziali sui parametri.
- $p(y \mid \theta)$ è la *verosimiglianza*, che collega i dati osservati ai parametri del modello.
- $p(\theta \mid y)$ è la *distribuzione a posteriori*, che rappresenta le credenze aggiornate sui parametri dopo aver osservato i dati.

La distribuzione a posteriori riassume il nostro stato di credenza sui possibili valori di $\theta$, aggiornato sulla base delle evidenze fornite dai dati.

Si noti che $p(y)$ non dipende dai parametri $\theta$. Pertanto, in molte situazioni pratiche, è sufficiente calcolare la distribuzione a posteriori fino a una costante. Per questo motivo, spesso la regola di Bayes viene riassunta come:

$$p(\theta \mid y) \propto p(y \mid \theta)p(\theta).$$

In questa forma, si ignora il denominatore poiché è una costante (indipendente da $\theta$). 

### Il Ruolo dei Priors

Una delle caratteristiche distintive dell'approccio bayesiano è l'incorporazione delle conoscenze a priori riguardo ai parametri del modello. Dichiarare questi priors ci obbliga a esplicitare tutte le assunzioni che facciamo sulla struttura del modello e sui suoi parametri. Allo stesso tempo, i priors sono spesso oggetto di critica nell'inferenza bayesiana a causa della soggettività che possono introdurre.

Tuttavia, l'inferenza bayesiana offre alcuni vantaggi meno evidenti a prima vista, tra cui:

- la capacità di lavorare efficacemente con piccoli set di dati,
- la capacità di eseguire la regolarizzazione del modello. 

Questi aspetti rendono l'approccio bayesiano particolarmente utile in situazioni in cui i dati sono scarsi o le assunzioni esplicite sul modello possono contribuire a migliorare le previsioni.

## Inferenza Predittiva

La distribuzione a posteriori dei parametri può essere utilizzata per modellare l'incertezza nelle previsioni $\tilde{y}$ relative a nuove osservazioni. La <font color='orange'>distribuzione predittiva a posteriori</font> di $\tilde{y}$ si ottiene marginalizzando la distribuzione congiunta delle previsioni $\tilde{y}$ e dei parametri $\theta$ rispetto ai parametri del modello:

$$
p(\tilde{y} \mid y) = \int p(\tilde{y}, \theta \mid y)d \theta = \int p(\tilde{y} \mid \theta, y)p(\theta|y)d\theta.
$$

In questo modo, la distribuzione predittiva può essere vista come una media delle previsioni del modello $p(\tilde{y} \mid \theta, y)$ ponderata sulla distribuzione a posteriori dei parametri del modello $p(\theta \mid y)$. Questo consente di incorporare l'incertezza sui parametri nel processo di previsione, rendendo le stime più robuste.

## Come possiamo eseguire l'inferenza bayesiana?

Esistono due approcci principali per determinare la distribuzione posteriore:

1. **Approccio Analitico (o Coniugato)**: Questo metodo è applicabile quando la distribuzione a priori scelta e la funzione di verosimiglianza appartengono alla stessa famiglia di distribuzioni, definite *coniugate*. In questi casi, la distribuzione posteriore può essere calcolata analiticamente, ovvero attraverso formule matematiche esatte. L'approccio coniugato è computazionalmente efficiente ma presenta una limitazione significativa: è applicabile solo in situazioni in cui si può assumere una coniugazione tra la distribuzione a priori e la verosimiglianza. Di conseguenza, trova un impiego limitato nelle analisi di dati reali, dove spesso le assunzioni di coniugazione risultano troppo restrittive.
   
2. **Approccio Numerico**: Quando non è possibile ottenere una forma analitica chiusa per la distribuzione posteriore, a causa della complessità del modello o della mancanza di coniugazione tra la distribuzione a priori e la verosimiglianza, si ricorre a metodi numerici. Questi algoritmi consentono di ottenere una stima approssimata, ma spesso accurata, della distribuzione posteriore.

### Metodi Numerici

Le catene di Markov Monte Carlo (MCMC) sono una classe di algoritmi ampiamente utilizzati in questo contesto. Essi costruiscono una catena di Markov che converge alla distribuzione posteriore desiderata. Tra i metodi MCMC più comuni troviamo:

- *Metropolis-Hastings*: Un algoritmo generale che consente di campionare da una vasta gamma di distribuzioni posteriori.
- *Gibbs Sampling*: Un caso particolare di Metropolis-Hastings, particolarmente efficiente quando la distribuzione congiunta è difficile da campionare direttamente, ma le distribuzioni condizionali sono note.

Oltre alle MCMC, esistono altre tecniche numeriche:

- *Variational Bayes*: Questo approccio consiste nel trovare la distribuzione $q(z)$ che meglio approssima la distribuzione posteriore $p(z \mid x)$, secondo un criterio di divergenza (ad esempio, la divergenza di Kullback-Leibler). L'obiettivo è trasformare il problema di inferenza esatta in un problema di ottimizzazione. Variational Bayes offre spesso una soluzione più veloce rispetto alle MCMC, ma l'approssimazione può essere meno accurata in alcuni casi.
- *Laplace approximation*: Questa tecnica consiste nell'approssimare la distribuzione posteriore con una distribuzione normale centrata sul massimo a posteriori (MAP) e con matrice di covarianza pari all'inverso della matrice di Hessiano negativa calcolata nel MAP. L'approssimazione di Laplace è computazionalmente efficiente, ma è valida solo localmente attorno al MAP e può portare a stime inaccurate della varianza posteriore.

Vantaggi: 

- Versatilità: L'approccio numerico è applicabile a una vasta gamma di modelli e distribuzioni. 
- Flessibilità: Consente di incorporare facilmente informazioni a priori complesse. 

Svantaggi:

- Costo computazionale: Può richiedere un tempo di calcolo considerevole, soprattutto per modelli complessi o grandi dataset.
- Tuning: La scelta dei parametri degli algoritmi MCMC (ad esempio, la proposta iniziale) può influenzare la convergenza e l'efficienza del campionamento.

In sintesi, l'approccio numerico offre una soluzione generale e flessibile per l'inferenza bayesiana, ma richiede una maggiore attenzione alla scelta degli algoritmi e alla valutazione della convergenza delle catene.

## Programmazione Probabilistica

I linguaggi di programmazione probabilistica (PPL) rappresentano un'importante innovazione nella modellazione bayesiana, facilitando l'uso di tecniche di approssimazione numerica per stimare le distribuzioni posteriori. Grazie ai PPL, la modellizzazione probabilistica diventa più accessibile, riducendo le barriere tecniche e computazionali. Questi strumenti consentono di definire modelli in modo dichiarativo, descrivendo le relazioni tra le variabili in termini probabilistici senza doversi occupare dei dettagli algoritmici sottostanti. In altre parole, i PPL permettono ai ricercatori di concentrarsi sull'espressione del modello, lasciando ai linguaggi il compito di gestire l'implementazione computazionale.

Tra i PPL più utilizzati troviamo:

- **Stan**: Uno dei linguaggi più popolari, noto per la sua efficienza e flessibilità.
- **PyMC**: Molto utilizzato nell'ecosistema Python, offre un'interfaccia user-friendly per la modellazione bayesiana.
- **TensorFlow**: Un framework che combina un approccio probabilistico con le reti neurali.

### Vantaggi della Programmazione Probabilistica in Psicologia

La programmazione probabilistica offre numerosi vantaggi per la ricerca psicologica, in particolare per l'analisi di processi complessi come l'apprendimento, le emozioni e il comportamento. Alcuni dei principali vantaggi includono:

- **Flessibilità**: I PPL, come Stan, Pyro, Numpyro, PyMC e Turing.jl, offrono un quadro flessibile per la definizione e la personalizzazione dei modelli probabilistici. In psicologia, questa flessibilità è cruciale, poiché i modelli devono adattarsi a una vasta gamma di processi mentali e comportamentali che variano tra individui e contesti.

- **Quantificazione dell'Incertezza**: La programmazione probabilistica permette di rappresentare esplicitamente e quantificare l'incertezza, un aspetto fondamentale in psicologia, dove molte variabili di interesse, come stati emotivi o atteggiamenti, sono latenti e soggette a incertezza. Incorporare questa incertezza nei modelli consente di ottenere stime più realistiche e affidabili.

- **Validazione del Modello**: I PPL facilitano la validazione dei modelli psicologici, consentendo ai ricercatori di confrontare le previsioni dei modelli con i dati osservati. Tecniche come i *posterior predictive checks* permettono di valutare la qualità e l'affidabilità del modello, contribuendo a una maggiore solidità delle conclusioni.

- **Modellazione Gerarchica**: Molti studi psicologici raccolgono dati a più livelli (ad esempio, misurazioni ripetute per individuo, sessioni sperimentali, contesti diversi). I PPL semplificano la costruzione e l'analisi di modelli gerarchici, catturando la variabilità sia intra- che inter-individuale.

- **Selezione e Confronto dei Modelli**: In psicologia è spesso necessario confrontare modelli con strutture diverse o ipotesi alternative. I PPL permettono di confrontare le capacità predittive dei modelli in modo sistematico e rigoroso, supportando la scelta del modello più adatto basandosi sull'accuratezza predittiva e non solo sulla complessità.

- **Comunicazione Trasparente**: La programmazione probabilistica favorisce la trasparenza nella modellizzazione. I ricercatori possono specificare chiaramente le assunzioni del modello, i *prior* e le funzioni di verosimiglianza, rendendo più facile la comunicazione e la collaborazione con altri esperti.

- **Librerie Estensibili**: I PPL offrono librerie estese e strumenti avanzati per lo sviluppo di modelli, l'inferenza e la visualizzazione. Questo riduce il carico computazionale e di implementazione, rendendo più agevole l'analisi di dati complessi tipici della psicologia sperimentale e clinica.

### Come Funzionano i PPL?

I linguaggi di programmazione probabilistica richiedono semplicemente la descrizione del modello probabilistico. Successivamente, utilizzano algoritmi di inferenza, come le catene di Markov Monte Carlo (MCMC) o l'inferenza variazionale, per stimare la distribuzione posteriore delle variabili di interesse. Ciò consente ai ricercatori di ottenere stime delle variabili sconosciute e di valutare l'incertezza associata.

In conclusione, i linguaggi di programmazione probabilistica hanno trasformato il modo in cui affrontiamo l'inferenza bayesiana, rendendola più accessibile e potente. Grazie alla loro semplicità d'uso e alla potenza computazionale, i PPL hanno reso l'inferenza bayesiana uno strumento sempre più diffuso in molte discipline, inclusa la psicologia. Questo approccio facilita la modellazione di fenomeni complessi e l'analisi rigorosa di dati, offrendo un metodo efficace per rispondere a domande di ricerca psicologica in modo trasparente e accurato.

## Notazione

In seguito, utilizzeremo $y$ per rappresentare i dati osservati e $\theta$ per indicare i parametri sconosciuti di un modello statistico. Entrambi, $y$ e $\theta$, saranno trattati come variabili casuali. Utilizzeremo $x$ per denotare le quantità note, come i predittori di un modello lineare.

È comune scrivere modelli statistici utilizzando la seguente notazione:

$$
\begin{aligned}
y & \sim \mathrm{normal}(\mu, \sigma) \\
\mu & \sim \mathrm{normal}(0, 10) \\
\sigma & \sim \mathrm{normal}^+(\sigma \mid  0, 1),
\end{aligned}
$$

dove il simbolo $\sim$ è chiamato *tilde* (`\sim` in LaTeX). 

In generale, possiamo leggere $\sim$ come *"è distribuito come"*, e questa notazione è usata come una scorciatoia per definire distribuzioni. L'esempio sopra può essere scritto anche come:

$$
\begin{aligned}
   p(y \mid \mu, \sigma) & = \mathrm{normal}(y \mid  \mu, \sigma)\\
   p(\mu) & = \mathrm{normal}(\mu \mid 0, 10)\\
   p(\sigma) & = \mathrm{normal}^+(\sigma \mid  0, 1).
\end{aligned}
$$

## Addendum: La Verosimiglianza Marginale

Nella discussione precedente, abbiamo introdotto la verosimiglianza marginale $p(y)$ come una costante di normalizzazione. Ma <font color='orange'>perché è così importante normalizzare la distribuzione posteriore?</font> La verosimiglianza marginale rappresenta la probabilità dei dati osservati, integrata su tutti i possibili valori del parametro $\theta$. In altre parole, esprime la probabilità di osservare i dati senza fare riferimento a un particolare valore del parametro.

Per comprendere intuitivamente, immagina di voler stimare la temperatura media di una stanza. Usando un termometro, ottieni una misurazione, ma sei consapevole che variabili come la posizione del termometro o l'ora del giorno potrebbero influenzare la lettura. La verosimiglianza marginale è equivalente a considerare la probabilità di ottenere una certa misurazione, prendendo in considerazione tutte queste possibili variabili.

Senza la normalizzazione, la somma delle probabilità assegnate ai diversi valori del parametro non sarebbe uguale a 1, il che significherebbe che non avremmo una distribuzione di probabilità valida. Questo renderebbe difficile interpretare correttamente i risultati. La verosimiglianza marginale agisce come una costante di normalizzazione, garantendo che l'area sotto la curva della distribuzione posteriore sia esattamente pari a 1, come richiesto da una distribuzione di probabilità.

La verosimiglianza marginale si calcola integrando (o sommando, nel caso di parametri discreti) la funzione di verosimiglianza rispetto a tutti i possibili valori del parametro, pesando ciascun valore con la sua probabilità a priori.

Consideriamo un esempio con una variabile casuale binomiale $Y$, la cui funzione di massa di probabilità (PMF) $p(Y)$ dipende dal parametro $\theta$. Supponiamo che $\theta$ possa assumere uno tra tre valori specifici: 0.1, 0.5 o 0.9, ciascuno con una probabilità a priori di $\frac{1}{3}$.

Se i dati indicano $n = 10$ prove e $k = 7$ successi, la funzione di verosimiglianza è data da:

$$
p(k = 7, n = 10 \mid \theta) = \binom{10}{7} \theta^7 (1 - \theta)^3.
$$

Per calcolare la verosimiglianza marginale $p(k = 7, n = 10)$, marginalizziamo su $\theta$, valutando la verosimiglianza per ciascun valore di $\theta$, moltiplicando per la probabilità a priori di ciascun $\theta$, e sommando i risultati:

$$
p(k = 7, n = 10) = \sum_{i=1}^{3} p(k = 7, n = 10 \mid \theta_i) \cdot p(\theta_i).
$$

Sostituendo i valori di $\theta$ e le probabilità corrispondenti:

$$
p(k = 7, n = 10) = \frac{1}{3} \binom{10}{7} 0.1^7 (1 - 0.1)^3 + \frac{1}{3} \binom{10}{7} 0.5^7 (1 - 0.5)^3 + \frac{1}{3} \binom{10}{7} 0.9^7 (1 - 0.9)^3.
$$

Questo calcolo dimostra come la marginalizzazione su $\theta$ incorpori tutte le sue possibili variazioni, ottenendo una stima complessiva che tiene conto dell'incertezza su $\theta$.

### Implementazione in Python

Per implementare questo calcolo in Python, possiamo definire una funzione per calcolare la verosimiglianza per i valori discreti di $\theta$ e poi sommare i risultati. Se invece i parametri sono continui, possiamo usare la libreria `scipy` per eseguire l'integrazione numerica.

Ecco un esempio di codice Python per il calcolo della verosimiglianza marginale nel caso discreto:

In [17]:
def likelihood_binomial(theta, k, n):
    return sp.comb(n, k) * (theta**k) * ((1 - theta) ** (n - k))


# Valori di theta e probabilità a priori
theta_values = [0.1, 0.5, 0.9]
prior = 1 / 3  # Probabilità a priori per ciascun theta
k = 7
n = 10

# Calcolo della verosimiglianza marginale
marginal_likelihood_discrete = sum(
    prior * likelihood_binomial(theta, k, n) for theta in theta_values
)

print(f"Likelihood Marginale (discreta): {marginal_likelihood_discrete}")

Likelihood Marginale (discreta): 0.05819729199999999


Per l'integrazione continua, si può utilizzare `scipy.integrate.quad`:

In [20]:
# Likelihood marginale su un intervallo continuo [0, 1]
marginal_likelihood_continuous, _ = quad(
    lambda theta: likelihood_binomial(theta, k, n), 0, 1
)

print(f"Likelihood Marginale (continua): {marginal_likelihood_continuous}")

Likelihood Marginale (continua): 0.09090909090909091


Nel codice precedente, utilizzando l'integrazione continua con `quad`, si calcola la verosimiglianza marginale integrando la funzione di verosimiglianza $p(k \mid \theta)$ rispetto a $\theta$ su quell'intervallo. In questo modo, implicitamente, si sta usando una <font color='orange'>distribuzione a priori uniforme</font> su $[0, 1]$. Questo perché la verosimiglianza non è stata moltiplicata per una funzione di densità a priori, quindi il peso che si assegna a ciascun valore di $\theta$ è lo stesso in tutto l'intervallo. La distribuzione uniforme assegna uguale probabilità a tutti i valori di $\theta$ tra 0 e 1, quindi si sta trattando $\theta$ come se provenisse da una distribuzione uniforme $\theta \sim \text{Uniform}(0, 1)$.

Se si desidera utilizzare una distribuzione a priori diversa (ad esempio, una Beta), è necessario moltiplicare la verosimiglianza per la densità di probabilità della distribuzione a priori prima di integrare. Ecco un esempio con una distribuzione Beta come prior:

In [23]:
# Definisci la distribuzione a priori come Beta(alpha, beta)
alpha_prior = 2
beta_prior = 2

# Likelihood marginale con prior Beta
marginal_likelihood_continuous_beta, _ = quad(
    lambda theta: likelihood_binomial(theta, k, n)
    * beta.pdf(theta, alpha_prior, beta_prior),
    0,
    1,
)

print(f"Likelihood Marginale con prior Beta: {marginal_likelihood_continuous_beta}")

Likelihood Marginale con prior Beta: 0.1118881118881119


In questo caso, il prior è una Beta(2,2), che dà più peso ai valori di $\theta$ vicini a 0.5, riflettendo una distribuzione a priori che preferisce valori intermedi di $\theta$.

## Riflessioni Conclusive

Al cuore della ricerca scientifica c'è una domanda del tipo: "dimmi qualcosa sulla variabile $\theta$ dato che ho osservato i dati $D$ e ho una certa conoscenza del meccanismo sottostante che genera i dati". La regola di Bayes fornisce la seguente risposta:

$$
p(\theta \mid D) = \frac{p(D \mid\theta) p(\theta)}{p(D)} = \frac{p(D \mid \theta) p(\theta)}{\int_\theta p(D \mid\theta) p(\theta) d\theta}.
$$

Questa equazione mostra come, partendo da un modello generativo $p(D \mid\theta)$ dei dati osservati e abbinato a una credenza a priori $p(\theta)$ su quali valori della variabile $\theta$ siano plausibili, possiamo inferire la distribuzione a posteriori $p(\theta \mid D)$ della variabile alla luce dei dati osservati.

La stima MAP (Massimo A Posteriori), che corrisponde al valore di $\theta$ che massimizza la distribuzione a posteriori, rappresenta una stima puntuale del parametro:

$$
\theta^* = \arg \max_\theta p(\theta \mid D).
$$

Nel caso di un prior non informativo (piatto), la stima MAP coincide con la stima di massima verosimiglianza, ovvero il valore di $\theta$ che massimizza la probabilità che il modello generi i dati osservati.

## Informazioni sull'Ambiente di Sviluppo {.unnumbered}

In [14]:
%load_ext watermark
%watermark -n -u -v -iv -w -m

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: Sat Oct 12 2024

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 24.0.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit

arviz     : 0.18.0
scipy     : 1.14.0
numpy     : 1.26.4
matplotlib: 3.9.1

Watermark: 2.4.3

